# Домашнее задние № 3, ч.1

### Выполнил: Кузнецов С.И.

Необходимо для каждой распознанной фамилии вывести на экран наиболее подходящий вариант из стартового протокола

In [1]:
import Levenshtein
import numpy as np

In [2]:
# что было написано в заявке

claim_list = [
    'Шехавцова Анна',
    'Гречихина Наталья',
    'Козлова Алена',
    'Груздева Алина',
    'Кущенко Анна',
    'Чистякова Анастасия'
]

In [3]:
# результат расшифровки речи диктора

speech_recognition = [
    'кучменко она',
    'кущенко оксана',
    'груздь алина',
    'рычихина наталья',
    'шиховцева на',
    'чистова анастасия'
]

Определим расстояние между именами.

In [4]:
def get_distance(Name1, Name2):
    '''
    Функция вычисляет расстояние между двумя именами Name1 и Name2.
    В качестве расстояния берем среднее расстояние Левинштейна между словами, входящими в Name1 и Name2.
    '''
    dists = []
    for n1, n2 in zip(Name1.lower().split(' '),Name2.lower().split(' ')):
        dists.append(Levenshtein.distance(n1, n2))
        
    return np.average(dists)

Создадим функцию, выбирающую наиболее близкое слово из списка.

In [5]:
def get_nearest_item(word, item, max_distance = None):
    '''
    Функция вычисляет наиболее близкое имя к слову word из списка item.
    Если в списке есть несколько наиболее близких слов, то выбирается то, у которого наиболее близкая фамилия.
    '''
    # Создадим список пар, вида ("Элемент списка": расстояние)
    distances = [(el, get_distance(el, word)) for el in item]
    distances.sort(key = lambda x: x[1])

    # Расстояние до ближайшего элемента в списке
    distance = distances[0][1]
    if max_distance:
        if distance > max_distance:
            return '<Не распознано>'

    # Список ближайших элементов
    nearest_names = [el[0] for el in distances if abs(el[1] - distance) < 1e-7]

    if len(nearest_names) == 1:
        return nearest_names[0]
    return get_nearest_item(word = ' '.join(word.split(' ')[1:]), item = nearest_names)
    

Каждой фамилии из списка поставим в соответствие наиболее близкую фамилию из протокола

In [6]:
for speech_name in speech_recognition:
    print('{}: {}'.format(speech_name, get_nearest_item(speech_name, claim_list)))

кучменко она: Кущенко Анна
кущенко оксана: Кущенко Анна
груздь алина: Груздева Алина
рычихина наталья: Гречихина Наталья
шиховцева на: Шехавцова Анна
чистова анастасия: Чистякова Анастасия


Имя "кущенко оксана" было распознано как "Кущенко Анна", поскольку Оксан в протокольном списке не было. 
Была Козлова Алена, которую не назвали. 
Чтобы избежать ситуации, когда имя не из списка распознается как имя в списке введем максимально домустимое 
расстояние между именами.

In [7]:
for speech_name in speech_recognition:
    print('{}: {}'.format(speech_name, get_nearest_item(speech_name, claim_list, max_distance = 2)))

кучменко она: Кущенко Анна
кущенко оксана: Кущенко Анна
груздь алина: Груздева Алина
рычихина наталья: Гречихина Наталья
шиховцева на: <Не распознано>
чистова анастасия: Чистякова Анастасия


In [8]:
for speech_name in speech_recognition:
    print('{}: {}'.format(speech_name, get_nearest_item(speech_name, claim_list, max_distance = 1.5)))

кучменко она: <Не распознано>
кущенко оксана: <Не распознано>
груздь алина: Груздева Алина
рычихина наталья: Гречихина Наталья
шиховцева на: <Не распознано>
чистова анастасия: Чистякова Анастасия


In [9]:
for speech_name in speech_recognition:
    print('{}: {}'.format(speech_name, get_nearest_item(speech_name, claim_list, max_distance = 3)))

кучменко она: Кущенко Анна
кущенко оксана: Кущенко Анна
груздь алина: Груздева Алина
рычихина наталья: Гречихина Наталья
шиховцева на: Шехавцова Анна
чистова анастасия: Чистякова Анастасия


При слишком маленьком значении расстояния не распознается большое количество имен, а при слишком большом возникают ложные распознавания.

Чтобы справиться с этой проблемой необходимо либо использовать другую метрику между словами, либо воспользоваться другим методом.